In [11]:
import prop
import pandas as pd
import matplotlib.pyplot as plt
from modules import pump, regen, heater, turbine, condenser

#Таблица с потоками и блоками:
streams = pd.read_excel("streams.xlsx",index_col=0)
blocks  = pd.read_excel("blocks.xlsx" ,index_col=0)

#Параметры нагревающей среды:
gas  = 'REFPROP::N2[0.7757]&O2[0.1068]&CO2[0.0193]&H2O[0.0936]&Ar[0.0046]' #Состав газа
Tin  = 183.6                                                               #Температура газов на входе
Pgas = 0.1                                                                 #Давление газов
Ggas = 509                                                                 #Расход газов
Tout = 80                                                                  #Температура газов на выходе

#Параметры охлаждающей среды:
fluidcond  = "REFPROP::WATER"  #Охлаждающая среда
Tfluidcond = 15                #Температура охл. среды
Pfluidcond = 0.5               #Давление охл. среды

#Параметры ОЦР:
fluid   = "REFPROP::AIR"    #Рабочая среда цикла
# Ppump   = 0.9                  #Давление на выходе из насоса
Pcond   = 0.1                #Давление в конденсаторе
KPDpump = 0.8                  #КПД насоса
KPDturb = 0.9                  #КПД турбины
DTheat  = 20                   #Минимальный напор в нагревателе
DTcond  = 5                    #Минимальный напор в конденсаторе



Ppump   = 0.5
for i in range(20):
    
    #Внесение входных данных в таблицу:
    streams.loc["IN-HEAT", "T":"Q"] = [Tin,Pgas,prop.t_p(Tin,Pgas,gas)["H"],prop.t_p(Tin,Pgas,gas)["S"],Ggas,1]
    streams.loc["IN-COND", "T":"Q"] = [Tfluidcond,Pfluidcond,prop.t_p(Tfluidcond,Pfluidcond,fluidcond)["H"],prop.t_p(Tfluidcond,Pfluidcond,fluidcond)["S"],1000,0]

    #Начало расчёта - задание расхода на первой итерации:
    G = Ggas*(prop.t_p(Tin,Pgas,gas)["H"]-prop.t_p(Tout,Pgas,gas)["H"])/(prop.t_p(Tin,Ppump,fluid)["H"]-prop.t_p(Tout,Ppump,fluid)["H"])

    #Поток перед насосом:
    streams.loc["COND-PUMP", "T":"Q"] = [prop.p_q(Pcond,0,fluid)["T"],Pcond,prop.p_q(Pcond,0,fluid)['H'],prop.p_q(Pcond,0,fluid)['S'],G,0]

    #Итеративный расчет для сведения баланса:
    for i in range(100):

        #Расчет цикла:
        pump.calc     ("COND-PUMP", "PUMP-REG",                          Ppump,  KPDpump,     fluid, streams, blocks)
        streams.loc["REG-HEAT", "T":"Q"] = streams.loc["PUMP-REG", "T":"Q"] #Игнор регенератора
        heater.calc   ("IN-HEAT",   "HEAT-OUT",  "REG-HEAT","HEAT-TURB", Tout,   DTheat, gas, fluid, streams, blocks)
        turbine.calc  ("HEAT-TURB", "TURB-REG",                          Pcond,  KPDturb,     fluid, streams, blocks)
        streams.loc["REG-COND", "T":"Q"] = streams.loc["TURB-REG", "T":"Q"] #Игнор регенератора
        condenser.calc("REG-COND",  "COND-PUMP", "IN-COND" ,"COND-OUT",  DTcond, fluidcond,   fluid, streams, blocks)

        #Тепловой баланс:
        Qbalance = blocks.loc['HEATER','Q']+blocks.loc['PUMP','N']-blocks.loc['CONDENSER','Q']-blocks.loc['TURBINE','N']
        print(i,round(Qbalance,5))

        #Выход из цикла при сведении баланса до требуемого уровня (в кВт):
        if abs(Qbalance) < 10**-2:
            print('+')
            break
    #КПД цикла:
    KPD = 100*(blocks.loc['TURBINE','N']-blocks.loc['PUMP','N'])/blocks.loc['HEATER','Q']
    Qturb = streams.loc["TURB-REG", "Q"]
    print('КПД цикла: ', round(KPD,3),'%', 'P=',Ppump, 'Q=',Qturb)

    Ppump   = Ppump+0.01


ValueError: HmolarPmolar: [PHFLSH error 1] One or more inputs are out of range; Temperature below triple-point or minimum temperature: T = 59.6878 K, Tmin = 59.7500 K.                                                                                                                 